# CS 109A/AC 209A/STAT 121A Data Science: Final Project
**Harvard University**<br>
**Fall 2016**<br>
**Instructors:** W. Pan, P. Protopapas, K. Rader<br>
**Members: ** Shawn Pan, Xinyuan (Amy) Wang, Ming-long Wu

## Remark - 052_quadratic_regression.ipynb ##
- Do quadratic regression models


In [37]:
import time
import pandas as pd
import numpy as np
import scipy as sp
from scipy.signal import savgol_filter
from scipy.stats import mode
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import dates
from matplotlib import colors
import matplotlib.dates as mdates
from datetime import datetime
from collections import Counter
from mpl_toolkits.basemap import Basemap
import statsmodels.api as sm
from sklearn import preprocessing
from collections import Counter
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
%matplotlib inline

### Load data

In [38]:
# Load listing data
listing = pd.read_csv('../../Data/data_after_predictor_selection.csv')
print(listing.columns.values)
listing.head()

['accommodates' 'bathrooms' 'bedrooms' 'review_scores_checkin'
 'review_scores_communication' 'latitude' 'longitude' 'property_type_0'
 'property_type_1' 'property_type_2' 'property_type_3' 'property_type_4'
 'property_type_5' 'room_type_0' 'room_type_1' 'room_type_2' 'bed_type_0'
 'bed_type_1' 'bed_type_2' 'bed_type_3' 'bed_type_4' 'beds'
 'review_scores_value' 'host_listing_count' 'review_scores_cleanliness'
 'review_scores_accuracy' 'minimum_nights' 'price']


,accommodates,bathrooms,bedrooms,review_scores_checkin,review_scores_communication,latitude,longitude,property_type_0,property_type_1,property_type_2,...,bed_type_2,bed_type_3,bed_type_4,beds,review_scores_value,host_listing_count,review_scores_cleanliness,review_scores_accuracy,minimum_nights,price
0,2,1.0,1.0,9.0,9.0,40.756852,-73.964754,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,9.0,1,7.0,9.0,3,160
1,10,1.0,3.0,9.0,8.0,40.830599,-73.941014,1.0,0.0,0.0,...,0.0,0.0,0.0,3.0,8.0,2,8.0,8.0,1,105
2,2,1.0,1.0,10.0,10.0,40.692189,-73.924120,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,10.0,4,10.0,10.0,3,58
3,2,1.0,1.0,10.0,10.0,40.734751,-74.002592,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,9.0,1,9.0,10.0,10,185
4,2,1.0,1.0,10.0,10.0,40.745282,-73.997836,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,10.0,1,10.0,10.0,1,195


## Feature engineering: location clustering to price by KNN ##

In [39]:
#Step to convert longitude and latitude to price cluster by KNN
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.neighbors import KNeighborsRegressor

class LocationPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = KNeighborsRegressor(n_neighbors=50, weights="uniform")
        super(LocationPreprocessor, self).__init__()
        
    def fit_transform(self, X, y):
        self.model.fit(X[["longitude", "latitude"]], y)
        return self.transform(X)
    
    def transform(self, X):
        loc = self.model.predict(X[["longitude", "latitude"]])
        Xnew = X.copy()
        Xnew["location"] = loc
        return Xnew

## Split data for training and testing data with transformation log(y)##

### Quadrature transformation of x###

In [40]:
#split numerical and categorical predictors
numr = ['accommodates', 'bathrooms', 'bedrooms', 'review_scores_checkin',
 'review_scores_communication', 'latitude', 'longitude', 'beds']
cate = ['property_type_0', 'property_type_1', 'property_type_2', 'property_type_3', 
        'property_type_4', 'property_type_5', 'room_type_0', 'room_type_1', 'room_type_2',
        'bed_type_0', 'bed_type_1', 'bed_type_2', 'bed_type_3', 'bed_type_4']
x_numr = listing[numr]
x_cate = listing[cate]
print x_numr.shape
print x_cate.shape

(27392, 8)
(27392, 14)


In [41]:
#quadracture transformation
quad_features = preprocessing.PolynomialFeatures(degree = 2)
x_expanded = quad_features.fit_transform(x_numr)
print x_expanded.shape

(27392, 45)


In [42]:
x = np.concatenate((x_expanded, x_cate.values), axis=1)
y = listing['price'].values
print x.shape
print y.shape

(27392, 59)
(27392,)


In [43]:
x_train, x_test, y_train, y_test = train_test_split(x, np.log(y), test_size=0.3, random_state=0)

## Linear Models: ##
### Linear regression ###

In [49]:
#fit linear regression model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

model = Pipeline([
            ("normalize", StandardScaler()),
            ("LinearModel", LinearRegression())
        ])

# model = Pipeline([
#             ("location", LocationPreprocessor()),
#             ("normalize", StandardScaler()),
#             ("LinearModel", LinearRegression())
#         ])

start_time = time.time()
model.fit(x_train, y_train)
compute_time = time.time() - start_time

print "Train Score", model.score(x_train, y_train)
print "Test Score", model.score(x_test, y_test)
LR_score = pd.Series([model.score(x_train, y_train), model.score(x_train, y_train), compute_time], 
                                                index=['Train R^2', 'Test R^2', 'Compute Time'])

Train Score 0.636819418513
Test Score 0.637551177773


### Lasso regression (L1 regularization) ###

In [45]:
#fit Lasso model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

model = Pipeline([
            ("normalize", StandardScaler()),
            ("lasso", Lasso(alpha=0.02))
        ])

start_time = time.time()
model.fit(x_train, y_train)
compute_time = time.time() - start_time

print "Train Score", model.score(x_train, y_train)
print "Test Score", model.score(x_test, y_test)
Lasso_score = pd.Series([model.score(x_train, y_train), model.score(x_train, y_train), compute_time], 
                                                index=['Train R^2', 'Test R^2', 'Compute Time'])

Train Score 0.564978386565
Test Score 0.573400270527


### Ridge regression (L2 regularization) ###

In [46]:
#fit Ridge model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

model = Pipeline([
            ("normalize", StandardScaler()),
            ("ridge", Ridge())
        ])

start_time = time.time()
model.fit(x_train, y_train)
compute_time = time.time() - start_time

print "Train Score", model.score(x_train, y_train)
print "Test Score", model.score(x_test, y_test)
Ridge_score = pd.Series([model.score(x_train, y_train), model.score(x_train, y_train), compute_time], 
                                                index=['Train R^2', 'Test R^2', 'Compute Time'])

Train Score 0.583849954339
Test Score 0.587212278547


### ElasticNet regression (L1 + L2 regularization) ###

In [47]:
#fit Ridge model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet

model = Pipeline([
            ("normalize", StandardScaler()),
            ("elasticnet", ElasticNet (alpha=0.02))
        ])

start_time = time.time()
model.fit(x_train, y_train)
compute_time = time.time() - start_time

print "Train Score", model.score(x_train, y_train)
print "Test Score", model.score(x_test, y_test)
ElasticNet_score = pd.Series([model.score(x_train, y_train), model.score(x_train, y_train), compute_time], 
                                                index=['Train R^2', 'Test R^2', 'Compute Time'])

Train Score 0.573240276619
Test Score 0.58090557328


### Compare four quadratic models ###

In [54]:
#compare all scores
score_df = pd.DataFrame({'Quad regression': LR_score, 
                         'Quad Lasso': Lasso_score,
                         'Quad Ridge': Ridge_score,
                         'Quad Elastic Net': ElasticNet_score})

score_df = score_df.reindex_axis(['Quad regression', 'Quad Lasso', 'Quad Ridge', 'Quad Elastic Net'], axis=1)
score_df

,Quad regression,Quad Lasso,Quad Ridge,Quad Elastic Net
Train R^2,0.636819,0.564978,0.58385,0.573240
Test R^2,0.636819,0.564978,0.58385,0.573240
Compute Time,0.037014,0.163459,0.02374,0.114789


In [55]:
# Save data
score_df.to_csv('../../Data/052_scores.csv', index=False)

### Conclusion: Quad regression without regularization performs best among four methods. ###